# Fase 0.4: Optimización del modelo en HAR (Hailo Archive)

In [ ]:
# Imports

import json
import os
import gc

import numpy as np
import tensorflow as tf
from IPython.display import SVG
from matplotlib import patches
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.python.eager.context import eager_mode
import torch
import onnx
import onnxruntime as ort
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from itertools import islice

# Funciones y parámetros de la CNN base
import sys
sys.path.append('../src/')
from cnn21_pix import *

# import the hailo sdk client relevant classes
from hailo_sdk_client import ClientRunner, InferenceContext

## 1. Optimización detallada

### 1.1. Carga de datos

In [ ]:
# Definir parámetros

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'

# Queremos usar todos los datos para la inferencia
SAMPLES=[0,0]
PAD=1
AUM=0

In [ ]:
# Carga de datos

(datos,H,V,B)=read_raw(DATASET)
(truth,H1,V1)=read_pgm(GT)

# Durante la ejecucion de la red vamos a coger patches de tamano cuadrado
sizex=32; sizey=32 

# Hacemos padding en el dataset para poder aprovechar hasta el borde
if(PAD):
    datos=torch.FloatTensor(np.pad(datos,((sizey//2,sizey//2),(sizex//2,sizex//2),(0,0)),'symmetric'))
    H=H+2*(sizex//2); V=V+2*(sizey//2)
    truth=np.reshape(truth,(-1,H1))
    truth=np.pad(truth,((sizey//2,sizey//2),(sizex//2,sizex//2)),'constant')
    H1=H1+2*(sizex//2); V1=V1+2*(sizey//2)
    truth=np.reshape(truth,(H1*V1))
    
# Necesitamos los datos en band-vector para hacer convoluciones
datos=np.transpose(datos,(2,0,1))

# Seleccionar conjunto de test (en este caso es una predicción)
(train,val,test,nclases,nclases_no_vacias)=select_training_samples(truth,H,V,sizex,sizey,SAMPLES)
dataset_test=HyperDataset(datos,truth,test,H,V,sizex,sizey)
print('  - test dataset:',len(dataset_test))

# Dataloader
batch_size=100 # defecto 100
# Indicamos shuffle = True porque no se evalúa todo el dataset
test_loader=DataLoader(dataset_test,batch_size,shuffle=True)

In [ ]:
# Cargar el modelo convertido a ONNX

ort_session = ort.InferenceSession("../results/models/model_cnn21.onnx")

# Cargar el modelo en HAR

model_name = "../results/models/model_cnn21"
hailo_model_har_name = f"{model_name}_hailo_model.har"
assert os.path.isfile(hailo_model_har_name), "Please provide valid path for HAR file"
runner = ClientRunner(har=hailo_model_har_name, hw_arch="hailo8l")

### 1.2 Evaluar el modelo en har sin optimización

In [ ]:
# Comprobacion rapida de la diferencia de precisión

# Crear tensor de entrada de prueba
input_tensor = torch.randn(1, B, sizex, sizey).to("cpu")
input_np = input_tensor.cpu().numpy()

# Realizar la inferencia en ONNX
output_onnx = ort_session.run(None, {'input': input_np})[0]

# print(output_onnx)

# Realizar la inferencia en HAR
with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
    input_har = np.transpose(input_np, (0, 2, 3, 1))

    # Realizar la inferencia en el modelo .har
    output_har = runner.infer(ctx, input_har)

    # print(output_har) 

# Comparar las diferencias
error = np.abs(output_onnx - output_har).mean()
print(f'Error medio entre ONNX y HAR: {error}')

In [ ]:
'''
# El bucle de inferencia tiene un memory leak que proviene del SDK de hailo con el emulador NATIVE
# Se realiza la evaluacion, por tanto, con un unico batch

# Inferencia

output=np.zeros(H*V,dtype=np.uint8)

# Medir el tiempo de ejecución
start_time = time.time()

# Realizar la predicción
with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
    total=0
    for (inputs, labels) in test_loader:
        # Convertir inputs a un formato adecuado para Hailo (numpy array)
        # Cambiamos de (batch_size, B, sizex, sizey) a (batch_size, sizey, sizex, B)
        inputs_np = np.transpose(inputs.numpy(), (0, 3, 2, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_np)
        
        predicted=np.argmax(native_res, axis=-1)
        predicted=predicted.squeeze()
        
        # Asignar las predicciones al array de salida
        for i in range(len(predicted)):
            output[test[total+i]]=np.uint8(predicted[i]+1)
        total+=labels.size(0)

        # Mostrar el progreso
        if(total%100000==0): print('  Test:',total,'/',len(dataset_test))
        
end_time = time.time()

print("Prediction time: {:.4f} seconds".format(end_time - start_time))

'''

In [ ]:
# Evaluar el modelo en HAR sin optimizacion

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100
data_iter = iter(test_loader)

for i in range(num_iters):

    inputs, labels = next(data_iter)
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

### 1.3 Aplicar modificaciones de optimización al modelo y evaluarlo

In [ ]:
# Crear un model script para el proceso de optimización

model_script_lines = [
    # Add normalization layer with mean [123.675, 116.28, 103.53] and std [58.395, 57.12, 57.375])
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # For multiple input nodes:
    # {normalization_layer_name_1} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_1_from_hn})\n',
    # {normalization_layer_name_2} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_2_from_hn})\n',
    # ...
]

# Load the model script to ClientRunner so it will be considered on optimization
runner.load_model_script("".join(model_script_lines))
runner.optimize_full_precision()

In [ ]:
# Evaluar el modelo en HAR con optimizacion

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100
data_iter = iter(test_loader)

for i in range(num_iters):

    inputs, labels = next(data_iter)
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_FP_OPTIMIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

### 1.4 Cuantizar el modelo y evaluarlo

In [ ]:
# Crear un dataset de calibración
# The original images are being used, just as the input to the SDK_FP_OPTIMIZED emulator
total_images = 1050

calib_dataset = np.zeros((total_images, sizex, sizey, B), dtype = np.float32)

count = 0

# Seleccionamos batches del dataloader para guardarlos en el dataset
for inputs, _ in test_loader:
    for img in inputs:
        if count >= total_images:
            break
            
        # Los inputs son de la forma (batch_size, B, sizex, sizey)
        img_np = img.numpy()
        # Trasponemos los inputs a formato (batch_size, sizex, sizey, B)
        img_har = np.transpose(img_np, (1, 2, 0))

        calib_dataset[count] = img_har
        count += 1
        
    if count >= total_images:
        break

In [ ]:
# Cuantizar el modelo con el dataset de calibración

# For calling Optimize, use the short version: runner.optimize(calib_dataset)
# A more general approach is being used here that works also with multiple input nodes.
# The calibration dataset could also be a dictionary with the format:
# {input_layer_name_1_from_hn: layer_1_calib_dataset, input_layer_name_2_from_hn: layer_2_calib_dataset}
hn_layers = runner.get_hn_dict()["layers"]
print("Input layers are: ")
print([layer for layer in hn_layers if hn_layers[layer]["type"] == "input_layer"])  # See available input layer names
calib_dataset_dict = {"model_cnn21/input_layer1": calib_dataset}  # In our case there is only one input layer

optimization_level = -100
compression_level = 0
# Mapeamos las proporciones de pesos de 4 bits según el nivel de compresión
compression_ratios = {
    0: 0.0,
    1: 0.2,
    2: 0.4,
    3: 0.6,
    4: 0.8,
    5: 1.0
}
auto_4bit_ratio = compression_ratios.get(compression_level, 0.0)

alls_lines = [
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # Batch size is 8 by default; 2 was used for stability on PCs with low amount of RAM / VRAM
    f"model_optimization_flavor(optimization_level={optimization_level}, compression_level={compression_level}, batch_size=8)\n",
    # The following line is needed because this is a really small model, and the compression_level is always reverted back to 0.'
    # To force using compression_level with small models, the following line should be used (compression level=4 equals to 80% 4-bit):
    f"model_optimization_config(compression_params, auto_4bit_weights_ratio={auto_4bit_ratio})\n",
    # The application of the compression could be seen by the [info] messages: "Assigning 4bit weight to layer .."
]

runner.load_model_script("".join(alls_lines))

runner.optimize(calib_dataset_dict)

In [ ]:
# Evaluar el modelo en HAR cuantizado

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100
data_iter = iter(test_loader)

for i in range(num_iters):

    inputs, labels = next(data_iter)
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_QUANTIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

In [ ]:
# Guardar el modelo cuantizado
# Let's save the runner's state to a Quantized HAR
quantized_model_har_path = f"{model_name}_quantized_model_o{optimization_level}_c{compression_level}.har"
runner.save_har(quantized_model_har_path)